# Same as Llama Notebook

## Installation of packages

In [1]:
# Required installations
!pip install transformers sentence-transformers faiss-cpu torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00


In [2]:
# Mount Google Drive (This may require manual approval for connection with Google Drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [3]:
import torch
from torch import cuda, bfloat16
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy as np
import os

## Defining paths to Google Drive

In [4]:
# Define paths for your Google Drive files
DRIVE_PATH = "/content/drive/MyDrive/DLProject"  # Base folder in Drive
INDEX_PATH = os.path.join(DRIVE_PATH, "qa_faiss_index.index")
PROCESSED_DATA_PATH = os.path.join(DRIVE_PATH, "processed_data.pkl")

# Verify files exist
if not os.path.exists(INDEX_PATH) or not os.path.exists(PROCESSED_DATA_PATH):
    raise FileNotFoundError("Required files not found in the specified directory.")

In [5]:
# Initialize model
print("Loading Qwen2.5-0.5B-Instruct...")
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

# If GPU is available it will run on
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
# Load embedding model
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# Load FAISS index
index = faiss.read_index(INDEX_PATH)
with open(PROCESSED_DATA_PATH, 'rb') as f:
    data = pickle.load(f)
    documents = data['documents']
    questions = data['questions']
print("Loaded")

Loading Qwen2.5-0.5B-Instruct...


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded


## RAG class

In [6]:
class RAGQuerySystem:
  """This is a class which is used for communication between the models. RAG foundation."""

  def __init__(self, index, documents, model, tokenizer, embeddings_model, questions):
      self.index = index
      self.documents = documents
      self.model = model
      self.tokenizer = tokenizer
      self.questions = questions
      self.embeddings_model = embeddings_model
      self.questions = questions


  def generate_answer(self, question, context, max_length=512):
      """This is a part where you can edit your prompt, context provides 3 most relevant informations from the vector store and question forwards the question"""

      prompt = f"""Answer using ONLY facts explicitly stated in this context:
      "{context}"

      If something isn't directly stated in the context, do not include it.
      Avoid repeating yourself.

      Question: "{question}"

      Answer what is directly stated in the context, additional information is not required."""
      # Create a list of messages for the chat template.
      # The 'system' message sets the persona of the assistant.
      # The 'user' message contains the actual prompt.
      messages = [
      {"role": "system", "content": "You are HU assistant for new students."},
      {"role": "user", "content": prompt} # This
      ]
      # Apply the chat template to format the messages for the Qwen model.
      text = tokenizer.apply_chat_template(messages,
                                           tokenize=False,
                                           add_generation_prompt=True)
      model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
      generated_ids = model.generate(**model_inputs,
                                     max_new_tokens=512)
      generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
      response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
      return response

  def query(self, question, k=3):

      # Get question embedding
      question_embedding = self.embeddings_model.encode([question], convert_to_tensor=True)
      if device != "cpu":
        question_embedding = question_embedding.cpu()

      # Search in FAISS
      distances, indices = self.index.search(question_embedding, k)

      # Get relevant documents
      relevant_docs = [f"Document {i+1}: {self.documents[i]}" for i in indices[0]]
      context = "\n\n".join(relevant_docs)

      # Get QA
      qa = [f"Q: {self.questions[i]} A: {self.documents[i]}" for i in indices[0]]

      # Generate answer
      answer = self.generate_answer(question, context)

      return {
          'answer': answer,
          'relevant_documents': relevant_docs,
          'distances': distances[0],
          'qa': qa
      }

  def clear_memory(self):
      torch.cuda.empty_cache()
      import gc
      gc.collect()

  def print_answer(self, query):
      result = rag_system.query(test_question)
      print(f"\n=== User Query ===\n{query}")
      print(f"\n=== Q&A ===")
      for (i, qa) in enumerate(result['qa']):
        print(f"{i+1}. {qa}")
      print(f"\n=== Generated Answer ===\n{result['answer']}")

In [7]:
# Initialize the RAG system
rag_system = RAGQuerySystem(index, documents, model, tokenizer, embeddings_model, questions)

In [8]:
def format_response(result):
    print("\n=== Generated Answer ===")
    print(result['answer'])
    print("\n=== Source Documents ===")
    for i, (doc, dist) in enumerate(zip(result['relevant_documents'], result['distances']), 1):
        print(f"\nSource {i} (Relevance Score: {1/(1+dist):.4f}):")
        print(doc)

# Interactive query function
def interactive_query(rag_system):
    while True:
        question = input("\nEnter your question (or 'quit' to exit): ")
        if question.lower() == 'quit':
            rag_system.clear_memory()
            break
        else:
            result = rag_system.query(question)
            format_response(result)

## Test cell

In [9]:
# Run test query
test_question = "What are the timings for the HU swimming pool?"
rag_system.print_answer(test_question)


=== User Query ===
What are the timings for the HU swimming pool?

=== Q&A ===
1. Q: What are the timings for male and female students to use the HU swimming pool? A: The HU swimming pool has separate schedules for male and female students:  Female Members: Monday, Wednesday, and Friday Male Members: Tuesday, Thursday, and Saturday
2. Q: What are the pool days and timings at Habib University? A: The pool is available on separate days for male and female students, with different timings:  Female: Monday, Wednesday, and Friday Male: Tuesday, Thursday, and Saturday  The pool timings are as follows:  Students: 11:00 AM – 05:00 PM Faculty: 11:30 AM – 07:30 PM Staff: 05:00 PM – 07:30 PM Alumni: 05:00 PM – 07:30 PM Working Saturday: 11:30 AM – 07:30 PM Non-working Saturday: 11:30 AM – 06:00 PM
3. Q: Are there any special fitness programs available at the HU swimming pool? A: Yes, the HU swimming pool offers special fitness programs for new swimmers. These programs focus on teaching basic swi

## Interactive Session


In [10]:
# Start interactive session
print("\n=== Starting Interactive Session ===")
print("Type your questions or 'quit' to exit")
interactive_query(rag_system)


=== Starting Interactive Session ===
Type your questions or 'quit' to exit

Enter your question (or 'quit' to exit): Where is Registrar's Office?

=== Generated Answer ===
The Registrar's Office is located in the Student Center at Habib University.

=== Source Documents ===

Source 1 (Relevance Score: 0.5286):
Document 3: The Registrar's Office is located in the Student Center at Habib University. You can visit them in person if you face any issues related to class schedules or enrollment.

Source 2 (Relevance Score: 0.4907):
Document 62: Submit a Change of Program/School Request Form to the Office of Academic Systems & Registrar.

Source 3 (Relevance Score: 0.4432):
Document 337: For technical support related to your HU email, ID card, or PeopleSoft, you can visit the User Computing and Information Technology Department located on the ground floor near the library. You can also reach out via email for assistance.

Enter your question (or 'quit' to exit): What are Swiming Pool Timings

In [11]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=11814d22235037f685cb98e5f9f4b63d68603021fd75dff70c56dd587b7db3f9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu
import os

# Download necessary NLTK data
nltk.download('punkt_tab') # Download 'punkt_tab' instead of just 'punkt'
# Evaluation

def calculate_bleu(generated_answer, expected_answer):
    """Calculates the BLEU score between a generated answer and an expected answer."""
    reference = nltk.word_tokenize(expected_answer)  # Tokenize the expected answer
    candidate = nltk.word_tokenize(generated_answer)  # Tokenize the generated answer
    return sentence_bleu([reference], candidate)  # Calculate BLEU score

# Load test data
with open(os.path.join(DRIVE_PATH, 'test_data_100.json'), 'r') as f:
    test_data = json.load(f)

# Initialize variables to store total and average BLEU scores
total_bleu = 0
average_bleu = 0

# Loop through test data, calculate BLEU score for each question-answer pair
for data_point in test_data:
    question = data_point['question']
    expected_answer = data_point['answer']
    result = rag_system.query(question)  # Replace with your RAG system object
    generated_answer = result['answer']

    # Calculate BLEU score and add to total
    bleu_score = calculate_bleu(generated_answer, expected_answer)
    total_bleu += bleu_score

# Calculate average BLEU score
average_bleu = total_bleu / len(test_data)

# Print the average BLEU score
print(f"Average BLEU Score: {average_bleu:.4f}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

Average BLEU Score: 0.1880


In [14]:
from rouge_score import rouge_scorer

In [15]:
# Load test data
with open(os.path.join(DRIVE_PATH, 'test_data_100.json'), 'r') as f:
    test_data = json.load(f)

# Initialize the RougeScorer object
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Initialize variable to store total ROUGE-1 scores
total_rouge_1_f1 = 0

# Loop through test data, calculate ROUGE score for each question-answer pair
for data_point in test_data:
    question = data_point['question']
    expected_answer = data_point['answer']
    result = rag_system.query(question)
    generated_answer = result['answer']

    # Calculate ROUGE scores
    scores = scorer.score(generated_answer, expected_answer)

    # Access and accumulate the ROUGE-1 F1 score
    rouge_1_f1_score = scores['rouge1'].fmeasure  # Access rouge1 instead of rougeL
    total_rouge_1_f1 += rouge_1_f1_score

# Calculate average ROUGE-1 F1 score
average_rouge_1_f1 = total_rouge_1_f1 / len(test_data)

# Print the average ROUGE-1 F1 score
print(f"Average ROUGE-1 Score: {average_rouge_1_f1:.4f}")



Average ROUGE-1 Score: 0.4661
